In [1]:
import torch
from torch.utils import data
# from torch.utils.data import DataLoader
import numpy as np
from utils import load_npz, read_ids
import os
import time

In [2]:
def load_npz(file_path):
    """
    Load data from a .npz file
    """
    with np.load(file_path) as data:
        X = data["X"]
        y = data["y"]
        # polygon_ids = data["polygon_ids"]
        block_ids = data["block_id"]
    return X, y, block_ids#, polygon_ids

In [3]:
L = 33
n_channel = 10

In [4]:
def standardize(X):
    m = X.mean(axis=0)
    s = X.std(axis=0)
    X = (X - m) / s
    return X

In [5]:
source_sits = "../../../data/theiaL2A_zip_img/output/2018/2018_SITS_data.npz"
train_val_eval = "../RF_model/train_val_eval_rf.txt"

train_ids, test_ids = read_ids(train_val_eval)
X_source, y_source, block_ids_source = load_npz(source_sits)
_source = np.concatenate((X_source, y_source[:, None], block_ids_source[:, None]), axis=1)
_source = _source[np.isin(_source[:, -1], train_ids)]
Xtrain_source = _source[:, :-2]
ytrain_source = _source[:, -2]
# Xtrain = np.concatenate((Xtrain_source, Xtrain_target), axis=0)
# ytrain = np.concatenate((ytrain_source, ytrain_target), axis=0)
Xtrain = Xtrain_source
ytrain = ytrain_source
x, y = Xtrain, ytrain
x = x.reshape(x.shape[0], L, n_channel)
x = x.transpose(0,2,1)
startime =time.time()
x_ = (standardize(x)).astype('float16')
print(time.time() - startime)
# x_ = x[idx]
# y = y.astype('int8')[idx]

53.99216985702515


In [7]:
# (x.mean(axis=0)).shape
# x.std(axis=0).astype('float16')

In [ ]:
# x_

In [34]:
x[0]

array([[ 319,  319,  319,  386,  454,  522,  437,  394,  310,  318,  326,
         468,  523,  617,  579,  538,  469,  421,  373,  356,  790,  538,
         525,  512,  458,  416,  375,  534,  488,  341,  366,  399,  399],
       [ 468,  468,  468,  583,  698,  814,  669,  596,  452,  471,  490,
         716,  790, 1013,  975,  913,  702,  653,  604,  551, 1362,  870,
         857,  845,  747,  661,  575,  883,  785,  556,  587,  609,  609],
       [ 301,  301,  301,  439,  578,  717,  490,  377,  151,  181,  212,
         584,  709, 1088, 1004,  871,  374,  331,  289,  286, 1221,  500,
         511,  522,  450,  367,  284,  607,  508,  261,  324,  489,  489],
       [ 187,  187,  187,  288,  389,  490,  294,  196,    0,   46,   93,
         478,  578, 1027,  874,  687,  332,  382,  433,  294,  980,  438,
         466,  495,  441,  359,  277,  503,  428,  158,  191,  305,  305],
       [  26,   26,   26,   17,    8,    0,    0,    0,    0,   23,   47,
         254,  310,  560,  388,  2

In [4]:
# source_sits = "../subset_data/2018_subset.npz"
# target_sits = "../subset_data/2019_subset.npz"
# train_val_eval = "train_val_eval_rf.txt"
# case = 1
# set_ = "trainval"
# # loader = data.DataLoader(dataset, batch_size = 10)
# transform = None
# import time
# st = time.time()
# dataset = SitsData(case, source_sits, target_sits, train_val_eval, set_, transform)
# loader = data.DataLoader(dataset, batch_size = 10)
# v, y = next(iter(loader))
# print("run time: ", st -time.time())

In [5]:
# loader = data.DataLoader(dataset, batch_size = 10)

In [6]:
# v, y = next(iter(loader))

In [8]:
# y.shape

In [7]:
class SITSData(data.Dataset):
    def __init__(self, case: int, source_sits, target_sits, train_val_eval, set_= 'trainval', transform=None):
        self.source_sits = source_sits
        self.target_sits = target_sits
        self.train_val_eval = train_val_eval
        self.transform = transform
        self.case = case
        self.set_ = set_

        # read the set ids
        start_time = time.time()
        self.train_ids, self.test_ids = read_ids(self.train_val_eval)
        print("read ids %s seconds ---" % (time.time() - start_time))
        start_time = time.time()
        # case selection
        if self.set_ == 'trainval':
            ids = self.train_ids
        elif self.set_ == 'test':
            ids = self.test_ids
        else:
            raise ValueError("Please choose a set between trainval and test")
        print("case %s seconds ---" % (time.time() - start_time))

        # read the data 
        start_time = time.time()
        X_source, y_source, block_ids_source = load_npz(self.source_sits)
        X_target, y_target, block_ids_target = load_npz(self.target_sits)
        print("load npz %s minutes ---" % ((time.time() - start_time) / 60))

        start_time = time.time()
        _source = np.concatenate((X_source, y_source[:, None], block_ids_source[:, None]), axis=1)
        _target = np.concatenate((X_target, y_target[:, None], block_ids_target[:, None]), axis=1)
        print("concatenate %s minutes ---" % ((time.time() - start_time) / 60))

        start_time = time.time()
        _source = _source[np.isin(_source[:, -1], ids)]
        _target = _target[np.isin(_target[:, -1], ids)]
        print("isin %s minutes ---" % ((time.time() - start_time) / 60))

        start_time = time.time()
        self.Xtrain_source = _source[:, :-2]
        self.ytrain_source = _source[:, -2]
        print("remove ids source %s seconds ---" % (time.time() - start_time))

        start_time = time.time()
        self.Xtrain_target = _target[:, :-2]
        self.ytrain_target = _target[:, -2]
        print("remove ids target %s seconds ---" % (time.time() - start_time))

        start_time = time.time()
        if self.case == 1:
            self.Xtrain = np.concatenate((self.Xtrain_source, self.Xtrain_target), axis=0)
            self.ytrain = np.concatenate((self.ytrain_source, self.ytrain_target), axis=0)
        elif self.case == 2:
            self.Xtrain = self.Xtrain_source
            self.ytrain = self.ytrain_source
        elif self.case == 3:
            self.Xtrain = self.Xtrain_target
            self.ytrain = self.ytrain_target
        else:
            raise ValueError("Please choose a case between 1 and 3") 
        print("concatenate %s minutes ---" % ((time.time() - start_time) / 60))

    def __len__(self):
        return len(self.ytrain)

    def __getitem__(self, idx):
        start_time = time.time()
        x, y = self.Xtrain, self.ytrain
        x = x.reshape(x.shape[0], L, n_channel)
        x = x.transpose(0,2,1)
        print("reshape and transpose %s seconds ---" % (time.time() - start_time))
        
        start_time = time.time()
        x = (standardize(x)).astype('float32')
        x_ = x[idx]
        y = y.astype('int8')[idx]
        print("stardardization %s seconds ---" % (time.time() - start_time))
        # start_time = time.time()
        # x_ = torch.from_numpy(np.array(x_))
        # y = torch.from_numpy(np.array(y))
        # print("to tenors %s seconds ---" % (time.time() - start_time))
        # return x, y
        return torch.from_numpy(np.array(x_)), torch.from_numpy(np.array(y))

def standardize(X):
    m = X.mean(axis=0)
    s = X.std(axis=0)
    X = (X - m) / s
    return X

In [10]:
# source_sits = "../../../data/theiaL2A_zip_img/output/2018/2018_SITS_data.npz"
# load_npz(source_sits)

In [8]:
# source_sits = "../subset_data/2018_subset.npz"
# target_sits = "../subset_data/2019_subset.npz"
# train_val_eval = "train_val_eval_rf.txt"
source_sits = "../../../data/theiaL2A_zip_img/output/2018/2018_SITS_data.npz"
target_sits = "../../../data/theiaL2A_zip_img/output/2019/2019_SITS_data.npz"
train_val_eval = "../RF_model/train_val_eval_rf.txt"
case = 1
set_ = "trainval"
transform = None
import time
start_time = time.time()
train_dataset = SITSData(case, source_sits, target_sits, train_val_eval, set_, transform)
train_loader = data.DataLoader(train_dataset, batch_size = 1, pin_memory=True)

print("run time: ", ((time.time() - start_time) / 60))

read ids 0.0021495819091796875 seconds ---
case 2.86102294921875e-06 seconds ---
load npz 1.9483068982760112 minutes ---
concatenate 0.12075953880945842 minutes ---
isin 0.21867297093073526 minutes ---
remove ids source 3.0279159545898438e-05 seconds ---
remove ids target 1.9788742065429688e-05 seconds ---
concatenate 0.06620307763417561 minutes ---
run time:  2.354695975780487


In [9]:
x_train,y_train= next(iter(train_loader))

reshape and transpose 2.5033950805664062e-05 seconds ---
stardardization 84.97270584106445 seconds ---


In [ ]:
for (x,y) in enumerate(train_loader):
    y_ry = y
    x_hr = x

reshape and transpose 4.029273986816406e-05 seconds ---
stardardization 82.94411087036133 seconds ---
reshape and transpose 3.147125244140625e-05 seconds ---
stardardization 82.9876160621643 seconds ---
reshape and transpose 4.673004150390625e-05 seconds ---


In [12]:
source_sits = "../../../data/theiaL2A_zip_img/output/2018/2018_SITS_data.npz"
target_sits = "../../../data/theiaL2A_zip_img/output/2019/2019_SITS_data.npz"
train_val_eval = "../RF_model/train_val_eval_rf.txt"
case = 1
set_ = "test"
transform = None
import time
st = time.time()
dataset_test = SITSData(case, source_sits, target_sits, train_val_eval, set_, transform)
test_loader = data.DataLoader(dataset_test, batch_size = 1, pin_memory=True)
# x_test,y_test= next(iter(loader))
print("run time: ", time.time() - st)

read ids 0.002400636672973633 seconds ---
case 1.9073486328125e-06 seconds ---
load npz 1.9369140625 minutes ---
concatenate 0.12013090848922729 minutes ---
isin 0.14379287560780843 minutes ---
remove ids source 4.100799560546875e-05 seconds ---
remove ids target 5.245208740234375e-06 seconds ---
concatenate 0.0441957155863444 minutes ---
run time:  134.75104451179504


In [15]:
start_time = time.time()
x_test,y_test= next(iter(test_loader))
print("run time: ", ((time.time() - start_time) / 60))

reshape and transpose 2.7179718017578125e-05 seconds ---
stardardization 54.03708863258362 seconds ---
to tenors 0.030365467071533203 seconds ---
run time:  0.9012147784233093


In [3]:
# # useful functions that are needed in the dataset class
# class SitsData(data.Dataset):
#     def __init__(self, case: int, source_sits, target_sits, train_val_eval, set_= 'trainval', transform=None):
#         super(SitsData, self).__init__()
#         self.source_sits = source_sits
#         self.target_sits = target_sits
#         self.train_val_eval = train_val_eval
#         self.transform = transform
#         self.case = case
#         self.set_ = set_
        
#         self.x, self.y = self._init_dataset()
        
#     def load_set(self, total_set):
#         """
#             Load a set of data:
#             to be used in prepare_data(), it organise dataset according to the set_name based block_ids
#             - set_name: train or test (reads from the block_ids using read_ids() utils) 
#             - total_set: the concatenated set of data for source or target
#             - returns X and Y:
#                 X is the data, Y is the label
#         """
#         self.train_ids, self.test_ids = read_ids(self.train_val_eval)

#         if self.set_ == "trainval":
#             ids = self.train_ids
#         elif self.set_ == "test":
#             ids = self.test_ids
#         else:
#             raise ValueError("Please choose a set between trainval and test")

#         set = total_set[np.isin(total_set[:, -1], ids)]
#         X = set[:, :-2]
#         Y = set[:, -2]
            
#         return X, Y
        
#     def _init_dataset(self):
#         """
#             Prepare data:
#             - load data from npz files(load_npz() in utils): returns X, Y and block_ids
#             - concatenate the data from source and target [X, Y, block_ids] == total_set
#             - split the data into train and test sets for source and target
#             - returns Xtrain and Ytrain base on CASE value
#             CASE: 
#             - 1: Train on Soucre and target, test on source and target
#             - 2: Train on Source only, test on Source and target
#             - 3: Train on Target only, test on Source and target
#         """
#             # print("Preparing data.........")

#         X_s, Y_s, block_ids_s = load_npz(self.source_sits)
#         X_t, Y_t, block_ids_t = load_npz(self.target_sits)
#         # print("Loading npz files done.........")
            

#         self.total_set_s = np.concatenate((X_s, Y_s[:, None], block_ids_s[:, None]), axis=1)
#         self.total_set_t = np.concatenate((X_t, Y_t[:, None], block_ids_t[:, None]), axis=1)
#             # print("Concatenating data done.........")

#             # Training set for target and source
#         self.Xtrain_s, self.Ytrain_s = self.load_set(self.total_set_s)
#         self.Xtrain_t, self.Ytrain_t = self.load_set(self.total_set_t)
#             # print("Loading train set done.........")

#             # Test set for target and source
#         self.Xtest_s, self.Ytest_s = self.load_set(self.total_set_s)
#         self.Xtest_t, self.Ytest_t = self.load_set(self.total_set_t)
#             # print("Loading test set done.........")

#         if self.case == 1:
#                 # concatenate training set for target and source
#             self.Xtrain = np.concatenate((self.Xtrain_s, self.Xtrain_t), axis=0)
#             self.Ytrain = np.concatenate((self.Ytrain_s, self.Ytrain_t), axis=0)

#         elif self.case == 2:
#                 # Xtrain is the training set for source only
#             self.Xtrain = self.Xtrain_s
#             self.Ytrain = self.Ytrain_s

#         elif self.case == 3:
#                 # Xtrain is the training set for target only
#             self.Xtrain = self.Xtrain_t
#             self.Ytrain = self.Ytrain_t

#         else:
#             raise ValueError("Please choose a case between 1 and 3")
            
#         return self.Xtrain, self.Ytrain
        
#     def __len__(self):
#         return len(self.y)
        
#     def __getitem__(self, idx):
#         x_, y_ = self.x[idx], self.y[idx]
            
#         x = torch.from_numpy(np.array(x_, dtype=int))
#         y = torch.from_numpy(np.array(y_, dtype=int))
#         return x, y
        

In [2]:
model_file= '../RF_model/models/rf_model_2.pkl'
ref_file = '../../../data/theiaL2A_zip_img/output/2018/2018_SITS_data.npz'
in_img = '../../../data/theiaL2A_zip_img/output/2018/2018_GapFilled_Image.tif'
out_path = '../../../results/RF'

In [3]:
import sys
import os
import joblib
import time
import csv
import optparse

import numpy as np

from osgeo import gdal, osr
from osgeo.gdalconst import *


def load_npz(file_path):
    """
    Load data from a .npz file
    """
    with np.load(file_path) as data:
        X = data["X"]
        y = data["y"]
       
    return X, y

def reshape_data(X, n_channel):
    """
    Reshape data to fit the model
    """
    X = X.reshape(X.shape[0], int(X.shape[1]/n_channel), n_channel)
    return X

#-----------------------------------------------------------------------
def reshape_data(X, nchannels):
	"""
		Reshaping (feature format (3 bands): d1.b1 d1.b2 d1.b3 d2.b1 d2.b2 d2.b3 ...)
		INPUT:
			-X: original feature vector ()
			-feature_strategy: used features (options: SB, NDVI, SB3feat)
			-nchannels: number of channels
		OUTPUT:
			-new_X: data in the good format for Keras models
	"""
	
	return X.reshape(X.shape[0],int(X.shape[1]/nchannels),nchannels) # x: row, y: time, z: band

#-----------------------------------------------------------------------
def read_minMaxVal(file):
	
	with open(file, 'r') as f:
		reader = csv.reader(f, delimiter=',')
		min_per = next(reader)
		max_per = next(reader)
	min_per = [float(k) for k in min_per]
	min_per = np.array(min_per)
	max_per = [float(k) for k in max_per]
	max_per = np.array(max_per)
	return min_per, max_per

#-----------------------------------------------------------------------
def save_minMaxVal(file, min_per, max_per):
	
	with open(file, 'w') as f:
		writer = csv.writer(f, delimiter=',')
		writer.writerow(min_per)
		writer.writerow(max_per)

#-----------------------------------------------------------------------
def computingMinMax(X, per=2):
	min_per = np.percentile(X, per, axis=(0,1))
	max_per = np.percentile(X, 100-per, axis=(0,1))
	return min_per, max_per

#-----------------------------------------------------------------------
def normalizingData(X, min_per, max_per):
	return (X-min_per)/(max_per-min_per)

def class_mapping(y_label):
	"""
	"""
	unique_class = np.unique(y_label)
	nclass = len(unique_class)
	max_ylabel = np.unique(y_label)[-1]+1 #-- +1 to take into account the case where y=0
	
	class_map = [0]*max_ylabel
	revert_class_map = unique_class.tolist()
	#-- Insert in class_map values from 1 to c, with c the number of classes
	n = nclass
	while n>0:
		insert_val = revert_class_map[n-1]
		class_map[insert_val] = n
		n = n-1	
	return class_map, revert_class_map

#-----------------------------------------------------------------------
def read_class_map(file):
	
	with open(file, 'r') as f:
		reader = csv.reader(f, delimiter=',')
		class_map = next(reader)
		revert_class_map = next(reader)
	class_map = [int(k) for k in class_map]
	revert_class_map = [int(k) for k in revert_class_map]
	return class_map, revert_class_map


def save_class_map(file, class_map, revert_class_map):
	
	with open(file, 'w') as f:
		writer = csv.writer(f, delimiter=',')
		writer.writerow(class_map)
		writer.writerow(revert_class_map)

nchannels = 10

# out_path = options.output
# model_file = options.model
# in_img = options.in_img # -i 54HWE_img.tif
# ref_file = options.ref_file #-t 54HWE_train.sqlite

model_name = model_file.split('/')[-1]
model_name = model_name.split('.')[0]

image_name = in_img.split('/')
image_name = image_name[-1].split('_')[0]

# out_map = out_path + '/' + image_name + '_' + model_name + '_map' + '.tif'
# print("out_map: ", out_map)
# if os.path.exists(out_map):
# 	print("out_map ",out_map,"already exists => exit")
# 	sys.exit("\n*** not overwriting out_map ***\n")

# out_confmap = out_path + '/' + image_name + '_' + model_name + '_proba' + '.tif'


model = joblib.load(model_file)

flag_del = False #-- deleting the training data
class_map_file = '.'.join(ref_file.split('.')[0:-1])
class_map_file = class_map_file + '_classMap.txt'
print("class_map_file: ", class_map_file)
if not os.path.exists(class_map_file): 
	X_train, y_train = load_npz(ref_file)
	class_map, revert_class_map = class_mapping(y_train)
	save_class_map(class_map_file, class_map, revert_class_map)
	flag_del = True
else:
	class_map, revert_class_map = read_class_map(class_map_file)

print("class_map: ", class_map)
print("revert_class_map: ", revert_class_map)

if flag_del:
	del X_train
	del y_train

#get image info about gps coordinates for origin plus size pixels
image = gdal.Open(in_img, gdal.GA_ReadOnly) #, NUM_THREADS=8
geotransform = image.GetGeoTransform()
originX = geotransform[0]
originY = geotransform[3]
spacingX = geotransform[1]
spacingY = geotransform[5]
r, c = image.RasterYSize, image.RasterXSize
out_raster_SRS = osr.SpatialReference()
out_raster_SRS.ImportFromWkt(image.GetProjectionRef())

print("r=", r, " -- c=", c)
print("originX: ", originX)
print("originY: ", originY)
print("spacingX: ", spacingX)
print("spacingY: ", spacingY)
print("geotransform: ", geotransform)

#-- Set up the characteristics of the output image
# driver = gdal.GetDriverByName('GTiff')
# out_map_raster = driver.Create(out_map, c, r, 1, gdal.GDT_Byte)
# out_map_raster.SetGeoTransform([originX, spacingX, 0, originY, 0, spacingY])
# out_map_raster.SetProjection(out_raster_SRS.ExportToWkt())
# out_map_band = out_map_raster.GetRasterBand(1)

# out_confmap_raster = driver.Create(out_confmap, c, r, 1, gdal.GDT_Float32)
# out_confmap_raster.SetGeoTransform([originX, spacingX, 0, originY, 0, spacingY])
# out_confmap_raster.SetProjection(out_raster_SRS.ExportToWkt())
# out_confmap_band = out_confmap_raster.GetRasterBand(1)


#convert gps corners into image (x,y)
# def gps_2_image_xy(x,y):
# 	return (x-originX)/spacingX,(y-originY)/spacingY
# def gps_2_image_p(point):
# 	return gps_2_image_xy(point[0],point[1])

size_areaX = 256
size_areaY = 256
x_vec = list(range(int(c/size_areaX)))
x_vec = [x*size_areaX for x in x_vec]
y_vec = list(range(int(r/size_areaY)))
y_vec = [y*size_areaY for y in y_vec]
x_vec.append(c)
y_vec.append(r)

proba_dist = []
for x in range(len(x_vec)-1):
	for y in range(len(y_vec)-1):
	
		xy_top_left = (x_vec[x],y_vec[y])
		xy_bottom_right = (x_vec[x+1],y_vec[y+1])
		
		print('top_left=',xy_top_left,' to bottom_right=',xy_bottom_right)

		#now loading associated data
		xoff = xy_top_left[0]
		yoff = xy_top_left[1]
		xsize = xy_bottom_right[0]-xy_top_left[0]
		ysize = xy_bottom_right[1]-xy_top_left[1]
		start_time = time.time()
		X_test = image.ReadAsArray(xoff=xoff, yoff=yoff, xsize=xsize, ysize=ysize) #, gdal.GDT_Float32
		print("Reading: ", time.time()-start_time)
        
        #-- reshape the cube in a column vector
		X_test = X_test.transpose((1,2,0))
		sX = X_test.shape[0]
		sY = X_test.shape[1]
		X_test = X_test.reshape(X_test.shape[0]*X_test.shape[1],X_test.shape[2])

		start_time = time.time()
		p_img = model.predict_proba(X_test)
    # proba_dist.append(p_img)
		proba_dist.append(p_img)

# 		# y_test = p_img.argmax(axis=1)
# 		y_prob = p_img.max(axis=1)

#         confpred_array = y_prob.reshape(sX,sY)
# 		# y_test = [revert_class_map[k] for k in y_test]
# 		# y_test = np.array(y_test, dtype=np.uint8)
# 		# pred_array = y_test.reshape(sX,sY)
		
# 		start_time = time.time()
# 		# out_map_band.WriteArray(pred_array, xoff=xoff, yoff=yoff)
#         out_confmap_band.WriteArray(confpred_array, xoff=xoff, yoff=yoff)
# 		print("Writing array: ", time.time()-start_time)

# 		start_time = time.time()
# 		out_map_band.FlushCache()
#         out_confmap_band.FlushCache()
# 		print("Writing disk: ", time.time()-start_time)

class_map_file:  ../../../data/theiaL2A_zip_img/output/2018/2018_SITS_data_classMap.txt
class_map:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 0, 11, 12, 13, 14, 15, 16, 17, 18, 0, 0, 0, 19]
revert_class_map:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 17, 18, 19, 23]
r= 10980  -- c= 10980
originX:  300000.0
originY:  4900020.0
spacingX:  10.0
spacingY:  -10.0
geotransform:  (300000.0, 10.0, 0.0, 4900020.0, 0.0, -10.0)
top_left= (0, 0)  to bottom_right= (256, 256)
Reading:  8.339349746704102
top_left= (0, 256)  to bottom_right= (256, 512)
Reading:  8.437856197357178
top_left= (0, 512)  to bottom_right= (256, 768)
Reading:  8.147621154785156
top_left= (0, 768)  to bottom_right= (256, 1024)
Reading:  8.552602052688599
top_left= (0, 1024)  to bottom_right= (256, 1280)
Reading:  8.632149457931519
top_left= (0, 1280)  to bottom_right= (256, 1536)
Reading:  8.985908031463623
top_left= (0, 1536)  to bottom_right= (256, 1792)
Reading:  8.284258127212524
top_left= (0, 1792)  to bottom_right= (

KeyboardInterrupt: 

In [4]:
probability_distribution = np.concatenate(proba_dist)

In [5]:
probability_distribution.shape

(57266176, 19)

In [6]:
(np.concatenate(proba_dist)).shape

(53217280, 19)

In [ ]:
# for x in range(len(x_vec)-1):
# 	for y in range(len(y_vec)-1):
	
# 		xy_top_left = (x_vec[x],y_vec[y])
# 		xy_bottom_right = (x_vec[x+1],y_vec[y+1])
		
# 		print('top_left=',xy_top_left,' to bottom_right=',xy_bottom_right)

# 		#now loading associated data
# 		xoff = xy_top_left[0]
# 		yoff = xy_top_left[1]
# 		xsize = xy_bottom_right[0]-xy_top_left[0]
# 		ysize = xy_bottom_right[1]-xy_top_left[1]
# 		start_time = time.time()
# 		X_test = image.ReadAsArray(xoff=xoff, yoff=yoff, xsize=xsize, ysize=ysize) #, gdal.GDT_Float32
# 		print("Reading: ", time.time()-start_time)

# 		X_test
# 		break

In [5]:
X_test.shape

(330, 256, 484)

In [6]:
X_test_ = X_test.transpose((1,2,0))
X_test_.shape

(256, 484, 330)

In [7]:
sX = X_test_.shape[0]
sY = X_test_.shape[1]
X_test_ = X_test_.reshape(X_test_.shape[0]*X_test_.shape[1],X_test_.shape[2])
p_img = model.predict_proba(X_test_)

In [ ]:
p_img

In [15]:
# hh = p_img.argmax(axis=1)

In [10]:
hs = p_img.max(axis=1)

In [17]:
hs.shape

(123904,)

In [16]:
n = hs.reshape(sX,sY)
n.

(256, 484)

In [28]:
hy = p_img + 0.2

In [29]:
# hy

In [30]:
print(np.linalg.norm(p_img - hy, axis=1))

[0.87177979 0.87177979 0.87177979 ... 0.87177979 0.87177979 0.87177979]
